---
# MLlib Spark

## Aprendizado de Máquina no Apache Spark

Alunos:


*   **Alisson Nunes** (725862)
*   **Lucas Mathaeus** (726561)
*   **William Eugênio** (726601)

Bacharelado em Ciência da Computação (BCC)<br>
Departamento de Computação (DC)<br>
Universidade Federal de São Carlos (UFSCar)


# (0) Contextualização

## Aprendizado de Máquina

- .
- .

## Classificação

## Clusterização

## Resilient Distributed Dataset (RDD)

#  (1) Instalação e Configuração

In [2]:
! pip install -q pyspark numpy

In [3]:
#configurando as variaveis de ambiente
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [38]:
#definindo uma sessão Spark para usar o pyspark
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

22/08/25 23:03:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/25 23:03:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
# Criação do contexto Spark
sc = ss.sparkContext

22/08/25 22:36:33 WARN Utils: Your hostname, monitora-Vostro-3583 resolves to a loopback address: 127.0.1.1; using 192.168.0.6 instead (on interface wlo1)
22/08/25 22:36:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/25 22:36:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/25 22:36:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/25 22:36:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# (2) Obtenção dos Dados

In [7]:
#instalando o módulo wget
#%%capture
! pip install -q wget
!rm -rf data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [8]:
# Baixando os dados das tabelas de dimensão do data mart Exame
import wget

url = "https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_binary_classification_data.txt"
wget.download(url, "data/sample_binary_classification_data.txt")

'data/sample_binary_classification_data (1).txt'

In [9]:
# Carregando o arquivo de dados
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "data/sample_binary_classification_data.txt")

# (3) Tratamento dos Dados

In [10]:
training, test = data.randomSplit([0.6, 0.4], seed=11)
training.cache()

PythonRDD[8] at RDD at PythonRDD.scala:53

In [11]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

model = LogisticRegressionWithLBFGS.train(training)

22/08/25 22:38:28 WARN Instrumentation: [8bfb2d8c] Initial coefficients will be ignored! Its dimensions (1, 692) did not match the expected size (1, 692)
22/08/25 22:38:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/08/25 22:38:28 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [12]:
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

In [13]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

metrics = BinaryClassificationMetrics(predictionAndLabels)

In [14]:
print("Area under PR = %s" % metrics.areaUnderPR)
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 1.0
Area under ROC = 1.0


In [19]:
print(predictionAndLabels.take(10))

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0)]


In [37]:
sc.stop()